In [4]:
#If error is showing while loading with pickle then first check pandas version
#pd.__version__ 
#replace with latest here I am installing version 1.4.1(any working version)
#%pip install pandas==1.4.1
import pandas as pd
import pickle
from scipy.stats import poisson

In [5]:
dict_table = pickle.load(open('dict_table','rb'))
df_historical_data = pd.read_csv('clean_fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('clean_fifa_worldcup_fixture.csv')

In [6]:
df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]

In [7]:
#now the fifa is played on neutral venue so we don't need to give weight to home or away. Thus I renamed them
#with simple team, GoalScored,....
df_home = df_home.rename(columns={'HomeTeam':'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

In [8]:
#now we concat them and then group by team and taking mean
df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


In [11]:
#Now lets write the poisson function for the prediction as we can see from the study in visulisation file
#nature of the prediction is Poisson.
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

In [12]:
#Now test the prediction between two teams
print(predict_points('England', 'Wales'))

(1.593887353091192, 1.0828229517246504)


In [13]:
#df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [14]:
# This time the group stage match is already over so this part is not needed
#for group in dict_table:
#    teams_in_group = dict_table[group]['Team'].values
#    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
#    for index, row in df_fixture_group_6.iterrows():
#        home, away = row['home'], row['away']
#        points_home, points_away = predict_points(home, away)
#        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
#        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

#    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
#    dict_table[group] = dict_table[group][['Team', 'Pts']]
#    dict_table[group] = dict_table[group].round(0)

#dict_table['Group A']

In [15]:
df_fixture_knockout

for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
48,Netherlands,Match 49,United States,2022,?
49,Argentina,Match 50,Australia,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Senegal,2022,?
52,Japan,Match 53,Croatia,2022,?
53,Brazil,Match 54,South Korea,2022,?
54,Morocco,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [16]:
#Now we apply predict the points and update the table
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated


get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,United States,2022,Netherlands
49,Argentina,Match 50,Australia,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Japan,Match 53,Croatia,2022,Croatia
53,Brazil,Match 54,South Korea,2022,Brazil
54,Morocco,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


In [17]:
# Now we apply the same for round of 8
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [18]:
#Now Update table and fixture
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,Croatia,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Spain,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [19]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Croatia,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Spain,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


In [20]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


In [21]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


In [22]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [23]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil


So as of today on 7th December the chance of Brazil to win Fifa-22. In my prediction the result of yesterdays match (which is played between Morocco and Spain ) result is not taken from data and Spain is declared as winner. Which is not fitting with my prdicted result.